In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun, tool
from langchain.tools.render import render_text_description_and_args, format_tool_to_openai_function
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder
from langchain.schema import ChatMessage, SystemMessage

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
)

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

# OPENAI_GPT_MODEL = "gpt-4-1106-preview"
OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"

In [2]:
import re

mp_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    max_retries=3,
    streaming=True
)

llm = ChatOpenAI(
    temperature=0.7,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=None,
    streaming=True
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = [
    MPThermoExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPElasticityExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPSummaryExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPDielectricExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPMagnetismExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    MPElectronicExpert(llm=mp_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=True)),
    # arxiv,
    # wikipedia,
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages[0].prompt.template = re.sub(
    r"\s+", " ",
    """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
    user to clarify their queries if needed. Please note that you don't have 
    direct control over MP but through multiple assistant agents to help you. 
    You need to provide complete context in the input for them to do their job.
    """).replace("\n", " ") + prompt.messages[0].prompt.template

prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm.bind(stop=["Observation"])
    # | map_reduce_chain  # TODO: Add map-reduce after LLM
    | JSONAgentOutputParser()
)


conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent_kwargs = {
    "extra_prompt_messages": [
        MessagesPlaceholder(variable_name="chat_history"),
        # SystemMessage(content=re.sub(
        #     r"\s+", " ",
        #     """You are a helpful data-aware agent that can consult materials-related
        #     data through Materials Project (MP) database, arXiv, and Wikipedia. Ask 
        #     user to clarify their queries if needed. Please note that you don't have 
        #     direct control to MP but through multiple assistant agents to help you. 
        #     You need to provide complete context for them to do their job. 
        #     """).replace("\n", " ")
        # )
        ],
    # "early_stopping_method": 'generate',
    # "extra_prompt_messages": 
    # )
}

agent_executor = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=5,
    memory=conversational_memory,
    # agent_kwargs=agent_kwargs,
    handle_parsing_errors=True,
)

# agent_executor = initialize_agent(
#     tools=tools,
#     llm=llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     max_iterations=5,
# )

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
agent_executor.invoke({
    "input": "What's the stiffest materials with the lowest formation energy in Si-O system?"
})



> Entering new AgentExecutor chain...
I can use the MPElasticityExpert tool to find the material with the highest Young's modulus and the MPThermoExpert tool to find the material with the lowest formation energy in the Si-O system. Let's start by finding the material with the highest Young's modulus using MPElasticityExpert.

Action:
```
{
  "action": "MPElasticityExpert",
  "action_input": "I am looking for the material with the highest Young's modulus in the Si-O system."
}
```



> Entering new AgentExecutor chain...


/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "chemsys": "Si-O",
    "sort_fields": "-young_modulus",
    "limit": 1,
    "fields": "material_id,formula_pretty,young_modulus"
  }
}```{"chemsys": "Si-O", "sort_fields": "-young_modulus", "limit": 1, "fields": "material_id,formula_pretty,young_modulus"}


Retrieving ElasticityDoc documents:   0%|          | 0/22 [00:00<?, ?it/s]

[{'formula_pretty': 'SiO2', 'material_id': 'mp-557837', 'elastic_tensor': {'raw': [[91.27281655572018, -12.554188118837994, -24.94828738859239, 34.007410213069306, -2.7755575615628907e-17, 2.498001805406602e-15], [-12.554188118837994, 82.85953528473897, 2.0499834965472976, 3.5917577405776444, 7.216449660063516e-16, -8.311618924685728e-16], [-24.94828738859239, 2.0499834965472976, 63.75899046702872, 6.710029101128894, 1.776356839400251e-15, 6.2068119236159814e-15], [34.007410213069306, 3.5917577405776444, 6.710029101128894, 37.63216075508616, 9.992007221626407e-16, 1.3183898417423732e-15], [-2.7755575615628907e-17, 7.216449660063516e-16, 1.776356839400251e-15, 9.992007221626407e-16, 42.96835504589667, 7.991003105240213], [2.498001805406602e-15, -8.311618924685728e-16, 6.2068119236159814e-15, 1.3183898417423732e-15, 7.991003105240213, 18.671476523966493]], 'ieee_format': [[75.0, -38.0, -0.0, 0.0, -6.0, -0.0], [-38.0, 91.0, 1.0, -0.0, -29.0, 0.0], [-0.0, 1.0, 76.0, -0.0, -8.0, -0.0], [0.0

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:230: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Action:
```{
  "action": "search_materials_thermo__get",
  "action_input": {
    "chemsys": "Si-O",
    "formation_energy_per_atom_min": 0,
    "sort_fields": "formation_energy_per_atom",
    "limit": 1,
    "fields": "material_id,formula_pretty,formation_energy_per_atom"
  }
}
```{"chemsys": "Si-O", "formation_energy_per_atom_min": 0.0, "sort_fields": "formation_energy_per_atom", "limit": 1, "fields": "material_id,formula_pretty,formation_energy_per_atom"}


Retrieving ThermoDoc documents:   0%|          | 0/8 [00:00<?, ?it/s]

[{'formula_pretty': 'SiO2', 'material_id': 'mp-555394', 'formation_energy_per_atom': 1.5152696604166664}]```
{
  "action": "Final Answer",
  "action_input": "The material with the lowest formation energy in the Si-O system is SiO2 with a formation energy per atom of 1.515 eV/atom (material_id: mp-555394)."
}
```

> Finished chain.

Observation: {'input': 'I am looking for the material with the lowest formation energy in the Si-O system.', 'output': 'The material with the lowest formation energy in the Si-O system is SiO2 with a formation energy per atom of 1.515 eV/atom (material_id: mp-555394).', 'intermediate_steps': [(AgentAction(tool='search_materials_thermo__get', tool_input={'chemsys': 'Si-O', 'formation_energy_per_atom_min': 0, 'sort_fields': 'formation_energy_per_atom', 'limit': 1, 'fields': 'material_id,formula_pretty,formation_energy_per_atom'}, log='Action:\n```{\n  "action": "search_materials_thermo__get",\n  "action_input": {\n    "chemsys": "Si-O",\n    "formation_energy_

{'input': "What's the stiffest materials with the lowest formation energy in Si-O system?",
 'chat_history': [],
 'output': "The material with the highest Young's modulus in the Si-O system is SiO2 with the material ID mp-557837. The material with the lowest formation energy in the Si-O system is also SiO2 with a formation energy per atom of 1.515 eV/atom (material_id: mp-555394).\n\nFinal Answer:\nThe stiffest material with the lowest formation energy in the Si-O system is SiO2."}